#### 0. Import basic methods (a.o. from source2.py)

In [1]:
from source2 import *
import numpy as np

In [44]:
# load basic info from the Input sheet
mapping = load_input_from_xls(filename=r'H2 Model - Input sheet.xlsm')
Inputs = extract_scenario(mapping, 'Scenario1')

##### 1. prepare input data and generate objects
We are testing the following case:

In [45]:
turbine_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Turbine')
turbine_data['depreciation_rate']=1/turbine_data['economic_lifetime']
display(turbine_data)

{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Turbine',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 1495000,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': 0.03333333333333333,
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.03,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.02,
 'residual_value': 0.01}

In [46]:
Turbine = Object(**turbine_data)
# hier heb je alleen het object met inputs

In [47]:
Turbine.generate_cashflows(startyear=2023, lifecycle=30, debug=False)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

In [48]:
foundation_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Foundation & cable')
foundation_data['depreciation_rate']==1/foundation_data['economic_lifetime']
display(foundation_data)

{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Foundation & cable',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 2691000,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': 0.03,
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.03,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.02,
 'residual_value': 0.01}

In [49]:
Foundation = Object(**foundation_data)
# hier heb je alleen het object met inputs

In [50]:
Foundation.generate_cashflows(startyear=2023, lifecycle=30, debug=False)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

#####  2. Test values - Turbines
##### 2.1 - CAPEX values

In [51]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914940000, -1866477600, -1903807151]}

In [52]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

CAPEX value testing in 2023: MTBS: -914940000 TUD/VO: -914940000.0
CAPEX value testing in 2024: MTBS: -1866477600 TUD/VO: -1866477600.0
CAPEX value testing in 2025: MTBS: -1903807151 TUD/VO: -1903807151.9999998


##### 2.2 - OPEX values

In [53]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053],
               'values': [-177500328.19374883, -181050334.75762382, -184671341.4527763, -188364768.28183183, -192132063.64746848, -195974704.92041785, -199894199.01882622, -203892082.99920273, -207969924.65918678, -212129323.1523705, -216371909.61541796, -220699347.80772632, -225113334.76388085, -229615601.45915848, -234207913.48834166, -238892071.75810847, -243669913.19327065, -248543311.4571361, -253514177.68627882, -258584461.2400044, -263756150.4648045, -269031273.4741006, -274411898.9435826, -279900136.92245424, -285498139.66090333, -291208102.4541214, -297032264.50320387, -302972909.7932679]}

In [54]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].opex.item(), 0)

Opex value testing in 2026: MTBS: -177500328.19374883 TUD/VO: -177500328.19374883
Opex value testing in 2027: MTBS: -181050334.75762382 TUD/VO: -181050334.75762382
Opex value testing in 2028: MTBS: -184671341.4527763 TUD/VO: -184671341.4527763
Opex value testing in 2029: MTBS: -188364768.28183183 TUD/VO: -188364768.28183183
Opex value testing in 2030: MTBS: -192132063.64746848 TUD/VO: -192132063.64746848
Opex value testing in 2031: MTBS: -195974704.92041785 TUD/VO: -195974704.92041785
Opex value testing in 2032: MTBS: -199894199.01882622 TUD/VO: -199894199.01882622
Opex value testing in 2033: MTBS: -203892082.99920273 TUD/VO: -203892082.99920273
Opex value testing in 2034: MTBS: -207969924.65918678 TUD/VO: -207969924.65918678
Opex value testing in 2035: MTBS: -212129323.1523705 TUD/VO: -212129323.1523705
Opex value testing in 2036: MTBS: -216371909.61541796 TUD/VO: -216371909.61541796
Opex value testing in 2037: MTBS: -220699347.80772632 TUD/VO: -220699347.80772632
Opex value testing i

##### 2.3 - Decommissioning


In [55]:
MTBS_values = {'years': [2053],
               'values': [-4685224752.0  * Turbine.decommissioning_rate * 1.847588815785422]}

In [56]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Decommissioning value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

Decommissioning value testing in 2053: MTBS: -173127377.02472454 TUD/VO: -173127377.02472454


##### 2.4 - Residual value

In [61]:
MTBS_values = {'years': [2053],
               'values': [865636885.1236223]}

In [62]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].revenue.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].revenue.item(), 0)

Residual value testing in 2053: MTBS: 865636885.1236223 TUD/VO: 865636885.1236223


#####  3. Test values - Foundations and cables
##### 3.1 - CAPEX values

In [64]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914940000, -1866477600, -1903807151]}

In [65]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

CAPEX value testing in 2023: MTBS: -914940000 TUD/VO: -1646892000.0


AssertionError: 
Arrays are not almost equal to 0 decimals
 ACTUAL: -914940000
 DESIRED: -1646892000.0

##### 3.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053],
               'values': [-177500328.19374883, -181050334.75762382, -184671341.4527763, -188364768.28183183, -192132063.64746848, -195974704.92041785, -199894199.01882622, -203892082.99920273, -207969924.65918678, -212129323.1523705, -216371909.61541796, -220699347.80772632, -225113334.76388085, -229615601.45915848, -234207913.48834166, -238892071.75810847, -243669913.19327065, -248543311.4571361, -253514177.68627882, -258584461.2400044, -263756150.4648045, -269031273.4741006, -274411898.9435826, -279900136.92245424, -285498139.66090333, -291208102.4541214, -297032264.50320387, -302972909.7932679]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

##### 3.3 - Decommissioning


In [ ]:
MTBS_values = {'years': [2053],
               'values': [-4685224752.0  * Turbine.decommissioning_rate * 1.847588815785422]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].revenue.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].revenue.item(), 0)

##### 2.4 - Residual value

In [ ]:
MTBS_values = {'years': [2053],
               'values': [865636885.1236223]}